In [ ]:
!pip install face_recognition pillow

In [ ]:
import face_recognition
from PIL import Image, ImageDraw


image_path1 = 'test1.jpg'
image_path2 = 'test2.jpg'

new_size = (472, 628)
threshold = 0.55

# 画像をリサイズする関数
def resize_image(image, size):
    return image.resize(size, Image.LANCZOS)

# 画像の読み込みと顔のエンコーディング
def encode_face(image_path):
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)
    if face_encodings:
        return image, face_encodings[0]  # 画像とエンコーディングの両方を返す
    else:
        return None, None



# 画像から顔をエンコードし、オリジナルの画像データも取得
image1, encoded_face1 = encode_face(image_path1)
image2, encoded_face2 = encode_face(image_path2)

if encoded_face1 is not None and encoded_face2 is not None:
    results = face_recognition.compare_faces([encoded_face1], encoded_face2)
    similarity = face_recognition.face_distance([encoded_face1], encoded_face2)

    if results[0] and similarity[0] < threshold:
        print("同一人物である可能性が高いです。")

    else:
        print("異なる人物である可能性が高いです。")

    similarity_percentage = (1 - similarity[0]) * 100
    print(f'類似度: {similarity_percentage:.1f}%')

    # ランドマークを抽出
    face_landmarks_list1 = face_recognition.face_landmarks(image1)
    face_landmarks_list2 = face_recognition.face_landmarks(image2)

    # PILイメージの作成
    pil_image1 = Image.fromarray(image1)
    pil_image2 = Image.fromarray(image2)

    # PILイメージ上にランドマークを描写
    draw1 = ImageDraw.Draw(pil_image1)
    for landmarks in face_landmarks_list1:
        for feature in landmarks.keys():
            draw1.line(landmarks[feature], width=5, fill='red')

    draw2 = ImageDraw.Draw(pil_image2)
    for landmarks in face_landmarks_list2:
        for feature in landmarks.keys():
            draw2.line(landmarks[feature], width=5, fill='red')

    # 画像をリサイズする
    pil_image1 = resize_image(pil_image1, new_size)
    pil_image2 = resize_image(pil_image2, new_size)

    # 修正された画像を横に並べる
    combined_image = Image.new('RGB', (new_size[0] * 2, new_size[1]))
    combined_image.paste(pil_image1, (0, 0))
    combined_image.paste(pil_image2, (new_size[0], 0))

    # 修正された画像を表示
    display(combined_image)
else:
    print("両画像の中に顔が認識できませんでした。")


